In [9]:
import numpy as np
import time

In [10]:
class SVGD:
    def __init__(self, bandwidth=None, step_size=1e-2):
        self.bandwidth = bandwidth
        self.step_size = step_size

    def _rbf_kernel(self, X):
        n, d = X.shape
        pairwise_dists = np.sum((X[:, None, :] - X[None, :, :]) ** 2, axis=2)
        h = self.bandwidth
        if h is None:
            h = np.median(pairwise_dists)
            h = 0.5 * h / np.log(n + 1)

        K = np.exp(-pairwise_dists / h)
        grad_K = -np.matmul(K, X) + X * np.sum(K, axis=1, keepdims=True)
        grad_K *= (2 / h)
        return K, grad_K

    def update(self, particles, grad_log_p):
        score = grad_log_p(particles)
        K, grad_K = self._rbf_kernel(particles)
        phi = (np.matmul(K, score) + grad_K) / particles.shape[0]
        return particles + self.step_size * phi

In [11]:
def log_posterior(x):
    """Log probability of a standard 2D Gaussian."""
    return -0.5 * np.sum(x**2, axis=1)

def grad_log_posterior(x):
    """Gradient of the log probability (standard Gaussian)."""
    return -x

In [12]:
def test_gradient_validity():
    particles = np.random.randn(100, 2)
    grad = grad_log_posterior(particles)
    assert grad.shape == particles.shape, "Gradient shape mismatch."
    assert not np.isnan(grad).any(), "Gradient contains NaNs."
    print("✅ Gradient shape and NaN check passed.")

In [13]:
def compute_kl(p_samples, log_q_fn):
    log_q = log_q_fn(p_samples)
    log_p = -0.5 * np.sum(p_samples ** 2, axis=1)
    return np.mean(log_p - log_q)

In [14]:
def test_kl_divergence_decreases():
    np.random.seed(42)
    svgd = SVGD()
    n_particles = 100
    dim = 2
    iterations = 300

    particles = np.random.randn(n_particles, dim)
    kl_values = []

    for i in range(iterations):
        particles = svgd.update(particles, grad_log_posterior)
        if i % 50 == 0:
            kl = compute_kl(particles, log_posterior)
            kl_values.append(kl)

    assert all(earlier >= later for earlier, later in zip(kl_values, kl_values[1:])), \
        f"KL did not decrease over time: {kl_values}"
    print("✅ KL divergence decreases test passed.")

In [15]:
def test_wall_time_under_limit():
    np.random.seed(0)
    svgd = SVGD()
    n_particles = 100
    dim = 2
    iterations = 1000
    time_budget_seconds = 20

    particles = np.random.randn(n_particles, dim)
    start_time = time.time()

    for _ in range(iterations):
        particles = svgd.update(particles, grad_log_posterior)

    elapsed = time.time() - start_time
    assert elapsed <= time_budget_seconds, f"Wall-time exceeded: {elapsed:.2f}s > {time_budget_seconds}s"
    print(f"✅ Wall-time test passed: {elapsed:.2f} seconds for {iterations} iterations")

In [16]:
if __name__ == "__main__":
    test_gradient_validity()
    test_kl_divergence_decreases()
    test_wall_time_under_limit()

✅ Gradient shape and NaN check passed.
✅ KL divergence decreases test passed.
✅ Wall-time test passed: 0.87 seconds for 1000 iterations
